In [1]:
import torch
import numpy as np

In [2]:
a = torch.FloatTensor(3,2)
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [3]:
a.zero_()

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In place operations have an underscore appended to their name. Operations without an underscore are the functional equivalent and create a copy of the tensor with the performed modification, leaving the original tensor untouched.

You can also create tensors from Python iterables.

In [4]:
torch.FloatTensor([[1,2,3], [3,2,1]])

tensor([[1., 2., 3.],
        [3., 2., 1.]])

In [5]:
n = np.zeros(shape=(3,2))
b = torch.tensor(n)
b

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]], dtype=torch.float64)

Notice that by default we created a 64 bit array. Usually in deep learning double precision in not required and it adds extra performance and memory overhead. Common practice is to use the 32 bit float type, or even the 16 bit float type, which is more than enough.

In [6]:
n = np.zeros(shape=(3,2), dtype=np.float32)
torch.tensor(n)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [7]:
n = np.zeros(shape=(3,2))
torch.tensor(n, dtype=torch.float32)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

Note that if you pass a `dtype` argument into the `torch.tensor` function it expected a `torch.float` dtype.

In [8]:
a = torch.tensor([1,2,3])
a

tensor([1, 2, 3])

In [9]:
# 0 dinemsional tensor.
s = a.sum()
s

tensor(6)

In [10]:
# Access the actual Python value of a 0 dimensional tensor using .item().
s.item()

6

In [11]:
torch.tensor(1)

tensor(1)

#### GPU Tensors

To convert from CPU to GPU, there is a tensor method `to(device)`, that creates a copy of the tensor to a specified device (CPU or GPU). 

In [12]:
a = torch.FloatTensor([2,3])
a

tensor([2., 3.])

In [13]:
ca = a.to('cuda')
ca

tensor([2., 3.], device='cuda:0')

In [14]:
a + 1

tensor([3., 4.])

In [15]:
ca + 1

tensor([3., 4.], device='cuda:0')

#### Gradients

Even with transparent GPU support, all of this dancing with tensors isn't worth
bothering with without one "killer feature"—the automatic computation of gradients. There are two approaches to how your gradients are calculated.

* **Static graph**: In this method, you need to define your calculations in advance and it won't be possible to change them later. The graph will be processed and optimized by the DL library before any computation is made.

* **Dynamic graph**: You don't need to define your graph in advance exactly as it will be executed, you just need to execute operations that you want to use for data transformation on your actual data. During this, the library will record the order of the operations performed, and when you ask it to calculate gradients, it will unroll its history of operations, accumulating the gradients of the network parameters. 

#### Tensors and Gradients

PyTorch tensors have a built-in gradient calculation and tracking system, so all you need to do is convert the data into tensors and perform computations using the tensor methods and functions provided by `torch`. There are several attributes related to gradients that every tensor has:

* grad: A property that holds a tensor of the same shape containing computed gradients.

* is_left:True if this tensor was constructed by the user and False if the object is a result of function transformation.

* requires_grad:True if this tensor requires gradients to be calculated. 

In [16]:
v1 = torch.tensor([1.0, 1.0], requires_grad=True)
v2 = torch.tensor([2.0, 2.0])

In [17]:
v_sum = v1 + v2
v_res = (v_sum*2).sum()
v_res

tensor(12., grad_fn=<SumBackward0>)

So now we have added both vectors element-wise, double every element, and then summed them together. The results is a 0 dimensional tensor with the value 12.

In [18]:
v1.is_leaf, v2.is_leaf

(True, True)

In [19]:
v_sum.is_leaf, v_res.is_leaf

(False, False)

In [20]:
v1.requires_grad, v2.requires_grad

(True, False)

In [21]:
v_sum.requires_grad, v_res.requires_grad

(True, True)

Now, let's tell PyTorch to calculate the gradients of our graph.

In [22]:
v_res.backward()
v1.grad

tensor([2., 2.])

By calling the `backward` function we asked PyTorch to calculate the numerical derivative of the `v_res` variable with respect to any variable that our graph has. In other words, what influence do small changes to the `v_res` variable have on the rest of the graph. In our particular example, the value of two in the gradients of `v1` means that by increasing any element of `v1` by one, the resulting value of `v_res` will grow by two.

As mentioned, PyTorch calculates gradients only for leaf tensors with requires `grad=True`. Indeed, if we try to check the gradients of v2, we get nothing:

In [23]:
v2.grad

#### Neural Network Building Blocks

In [24]:
import torch.nn as nn
l = nn.Linear(2, 5)
v = torch.FloatTensor([1, 2])
l(v)

tensor([ 0.2722, -0.0176, -0.9862,  0.6233, -1.1733], grad_fn=<AddBackward0>)

Here, we created a randomly initialized feed-forward layer, with two inputs and 5 outputs, and applied it to our float tensor. All classes in the `torch.nn` packages inherit from the nn.Module base class, which you can use to implement your own higher-level NN blocks. You will see how you can do this in the next section, but, for now, let's look at useful methods that all `nn.Module` children provide. They are
as follows:

* parameters(): This function returns an iterator of all variables that require gradient computation (that is, module weights).
* zero_grad(): This function initializes all gradients of all parameters to zero.
* to(device): This function moves all module parameters to a given device (CPU or GPU).
* state_dict(): This function returns the dictionary with all module parameters and is useful for model serialization.
* load_state_dict(): This function initialized the module with the state dictionary.

`Sequential` is a convenient class that allows you to combine other layers into a pipeline.

In [25]:
s = nn.Sequential(
nn.Linear(2, 5),
nn.ReLU(),
nn.Linear(5, 20),
nn.ReLU(),
nn.Linear(20, 10),
nn.Dropout(p=0.3),
nn.Softmax(dim=1))

s

Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=20, bias=True)
  (3): ReLU()
  (4): Linear(in_features=20, out_features=10, bias=True)
  (5): Dropout(p=0.3, inplace=False)
  (6): Softmax(dim=1)
)

In [26]:
s(torch.FloatTensor([[1, 2]]))

tensor([[0.0686, 0.0420, 0.0552, 0.0889, 0.2062, 0.0692, 0.1556, 0.1728, 0.0779,
         0.0637]], grad_fn=<SoftmaxBackward>)

#### Custom Layers

By subclassing the `nn.Module` class, you can create your own building blocks, which can be stacked together, reused later, and integrated into the PyTorch framework flawlessly. At its core, the `nn.Module provides` quite rich functionality to its children:

* It tracks all submodules that the current module includes. For example, your building blocks can have two feed-forward layers used somehow to perform the block's transformation.

* It provides functions to deal with all parameters of the registered submodules. You can obtain a full list of the modules parameters (`parameters()` method), zero its gradients (`zero_grads()` method), serialize and deserialize the module (`state_dict()` and `load_state_dict()`) and even perform generic transformations using your own callable (`apply()` method).

* It establishes the convention of `Module` application to data. Every module needs to perform its data transformation in the `forward()` method by overriding it. 

* There are some more functions, such as the ability to register a hook function to tweak modeul transformation or gradients flow, but they are more for advanced use cases.

To make our life simpler, when following the preceding convention, PyTorch authors simplified the creation of modules through careful design and a good dose of Python magic. So, to create a custom module, we usually have to do only two things—register submodules and implement the `forward()` method. Let's look at how this can be done for our Sequential example from the previous
section, but in a more generic and reusable way.

In [27]:
class OurModule(nn.Module):
    def __init__(self, num_inputs, num_classes, dropout_prob=0.3):
        super(OurModule, self).__init__()
        self.pipe = nn.Sequential(
            nn.Linear(num_inputs, 5),
            nn.ReLU(),
            nn.Linear(5, 20),
            nn.ReLU(),
            nn.Linear(20, num_classes),
            nn.Dropout(p=dropout_prob),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        return self.pipe(x)

This is our module class that inherits ``nn.Module``. In the constructor, we pass three parameters: the input size, the output size, and the optional dropout probability. The first thing we need to do is call the parent's constructor to let it initialize itself.

In [28]:
def forward(self, x):
    return self.pipe(x)

Here, we must override the `forward` function with our implementation of the data transformation. As our module is a very simple wrapper around other layers, we just need to ask them to transform the data. 

In [29]:
net = OurModule(num_inputs=2, num_classes=3)
v = torch.FloatTensor([[2, 3]])
out = net(v)
print(net)
print(out)

OurModule(
  (pipe): Sequential(
    (0): Linear(in_features=2, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
    (5): Dropout(p=0.3, inplace=False)
    (6): Softmax(dim=1)
  )
)
tensor([[0.3422, 0.3497, 0.3081]], grad_fn=<SoftmaxBackward>)


#### The final glue - loss functions and optimizers

#### Loss Functions

Loss functions reside in the `nn` package and are implement as an `nn.Module` subclass. Usually, they accept two arguments: output from the network and desired output. The most commonly used standard loss functions are:

* `nn.MSELoss`
* `nn.BCELoss` and `nn.BCEWithLogits`: Binary cross-entropy loss. The first version expects a single probability value (usually it's the output of the `Sigmoid` layer), while the second version assumes raw scores as input and applies `Sigmoid` itself. The second is usually more stable and efficient. 
* `nn.CrossEntropyLoss` and `nn.NLLLoss`: Famous maximum likelihoos criteria that are used in multi-class classification problems. The first version expects raw scores for each class and applies `LogSoftmax` internally, while the second expects to have log probabilities as the input.

#### Optimizers

The resposibility of the basic optimizer is to take the gradients of model parameters and change these parameters in order to decrease the loss value. In the `torch.optim` package, PyTorch provides lots of popular optimizer implementations and the most widely known are as follows:

* SGD: A vanilla stochastic gradient descent algorithm with an optional momentum extension.
* RMSprop
* Adagrad
* Adam

On construction, you need to pass an interable of tensors, which will be modified during the optimization process. The usual practice is to pass the result of the `params()` call of the upper level `nn.Module` instance, which will return an interable of all leaf tensors with gradients. Now, let's discuss the common blueprint of a training loop.

In [ ]:
for batch_x, batch_y in iterate_batches(data, batch_size=32):
    batch_x_t = torch.tensor(batch_x)
    batch_y_t = torch.tensor(batch_y)
    out_t = net(batch_x_t)
    loss_t = loss_function(out_y, batch_y_t)
    loss_t.backward()
    optimizer.step()
    optimizer.zero_grad()

#### Monitoring with TensorBoard